## Visualization

In [1]:
import numpy as np 
import pandas as pd
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv("data/alldata.csv", index_col = 0)

C:\Users\pixin\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
data.index = np.arange(len(data))

In [6]:
data = data.drop("Unnamed: 0.1", 1)

In [7]:
hier = np.vectorize(lambda x : 1 if "-" in x else len(x) - 1)
data["level"] = hier(data["NAICS.id"])

In [8]:
len(data["NAICS.display-label"].unique())

105

In [9]:
level1_name = set(data["NAICS.display-label"][data["level"] == 1])
level2_name = set(data["NAICS.display-label"][data["level"] == 2])
level3_name = set(data["NAICS.display-label"][data["level"] == 3])
level4_name = set(data["NAICS.display-label"][data["level"] == 4])
level5_name = set(data["NAICS.display-label"][data["level"] == 5])
level1_id = set(data["NAICS.id"][data["level"] == 1])
level2_id = set(data["NAICS.id"][data["level"] == 2])
level3_id = set(data["NAICS.id"][data["level"] == 3])
level4_id = set(data["NAICS.id"][data["level"] == 4])
level5_id = set(data["NAICS.id"][data["level"] == 5])

In [10]:
append0 = np.vectorize(lambda x : str(x) if len(str(x)) == 5 else "0" + str(x))
data["zip"] = append0(data["zip"])

In [11]:
plt.bar(5-np.arange(len(data["level"].value_counts())), data["level"].value_counts())

<BarContainer object of 5 artists>

In [12]:
# data["level"].value_counts()
data.columns

Index(['zip', 'NAICS.id', 'NAICS.display-label', 'RCPSZFE.display-label',
       'ESTAB', 'City Name', 'State', 'level'],
      dtype='object')

In [13]:
inter_45 = level4_name.intersection(level5_name)
inter_34 = level3_name.intersection(level4_name)
inter_23 = level2_name.intersection(level3_name)
r = np.arange(len(data))
l5 = data[data["level"] == 5]

In [14]:
data.loc[16, "NAICS.display-label"]
get_labels = np.vectorize(lambda x : x in inter_45)
labels = get_labels(l5["NAICS.display-label"])
idx2drop = l5.index[labels]

In [15]:
data = data.drop(idx2drop,0) 
data.index = np.arange(len(data))

In [16]:
l4 = data[data["level"] == 4]
get_labels = np.vectorize(lambda x : x in inter_34)
labels = get_labels(l4["NAICS.display-label"])
idx2drop = l4.index[labels]
data = data.drop(idx2drop,0) 
data.index = np.arange(len(data))

In [17]:
l3 = data[data["level"] == 3]
get_labels = np.vectorize(lambda x : x in inter_23)
labels = get_labels(l3["NAICS.display-label"])
idx2drop = l3.index[labels]
data = data.drop(idx2drop,0) 
data.index = np.arange(len(data))

In [18]:
level1_name = set(data["NAICS.display-label"][data["level"] == 1])
level2_name = set(data["NAICS.display-label"][data["level"] == 2])
level3_name = set(data["NAICS.display-label"][data["level"] == 3])
level4_name = set(data["NAICS.display-label"][data["level"] == 4])
level5_name = set(data["NAICS.display-label"][data["level"] == 5])
level1_id = set(data["NAICS.id"][data["level"] == 1])
level2_id = set(data["NAICS.id"][data["level"] == 2])
level3_id = set(data["NAICS.id"][data["level"] == 3])
level4_id = set(data["NAICS.id"][data["level"] == 4])
level5_id = set(data["NAICS.id"][data["level"] == 5])

In [19]:
labels = list(data["RCPSZFE.display-label"] == 'Establishments not operated for the entire year')
idx2drop = data.index[labels]
data = data.drop(idx2drop,0) 
data.index = np.arange(len(data))

In [20]:
labels = list(data["RCPSZFE.display-label"] ==  'Establishments operated for the entire year')
idx2drop = data.index[labels]
data = data.drop(idx2drop,0) 
data.index = np.arange(len(data))

In [21]:
values = data["RCPSZFE.display-label"].unique()
values

array(['All establishments',
       'Establishments operated entire year with sales/receipts/revenue of $100,000 to $249,999',
       'Establishments operated entire year with sales/receipts/revenue of $250,000 to $499,999',
       'Establishments operated entire year with sales/receipts/revenue of $500,000 to $999,999',
       'Establishments operated entire year with sales/receipts/revenue of $1,000,000 or more',
       'Establishments operated entire year with sales/receipts/revenue less than $100,000'],
      dtype=object)

In [22]:
weight_dict = {'All establishments' : 0,
          'Establishments operated entire year with sales/receipts/revenue less than $100,000' : 0.5,
          'Establishments operated entire year with sales/receipts/revenue of $100,000 to $249,999': 1,
          'Establishments operated entire year with sales/receipts/revenue of $250,000 to $499,999' : 2,
          'Establishments operated entire year with sales/receipts/revenue of $500,000 to $999,999' : 4,
          'Establishments operated entire year with sales/receipts/revenue of $1,000,000 or more' : 8
         }

In [23]:
scale_dict = {'All establishments' : "Total",
          'Establishments operated entire year with sales/receipts/revenue less than $100,000' : "E",
          'Establishments operated entire year with sales/receipts/revenue of $100,000 to $249,999': "D",
          'Establishments operated entire year with sales/receipts/revenue of $250,000 to $499,999' : "C",
          'Establishments operated entire year with sales/receipts/revenue of $500,000 to $999,999' : "B",
          'Establishments operated entire year with sales/receipts/revenue of $1,000,000 or more' : "A"
         }

In [24]:
give_weight = np.vectorize(lambda x : weight_dict[x])
data['weight'] = give_weight(data["RCPSZFE.display-label"])

In [25]:
give_cat = np.vectorize(lambda x : scale_dict[x])
data['scale'] = give_cat(data["RCPSZFE.display-label"])

In [26]:
temp = data[data["scale"] == "Total"]
temp = temp[temp["level"] == 1]["ESTAB"]
# # plt.hist(temp)
dist = dict(temp.value_counts())
l = sorted(dist.items(), key = lambda x : x[0])
num = [pair[0] for pair in l]
count = [pair[1] for pair in l]
plt.bar(x=num, height=count)

<BarContainer object of 405 artists>

## Cumulative Density Function for establishments' counting

In [27]:
# print(np.array(dist.values))
# count
print(num)
y = np.cumsum(count)
cdf = y / y[-1]
plt.plot(num, cdf)
cdf

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

array([0.17357104, 0.27311761, 0.33738664, 0.38736603, 0.42271228,
       0.45201292, 0.47619538, 0.49742374, 0.51490794, 0.53115554,
       0.54633828, 0.55987222, 0.57189475, 0.58223413, 0.59271091,
       0.60167628, 0.61064166, 0.61946963, 0.62816021, 0.63557983,
       0.64293075, 0.64880462, 0.65546854, 0.6623042 , 0.66876202,
       0.67539159, 0.68150591, 0.68686452, 0.69212009, 0.6968604 ,
       0.70184117, 0.70719978, 0.71204314, 0.71657736, 0.72090547,
       0.72526793, 0.72952734, 0.73471421, 0.73797747, 0.74192773,
       0.74560319, 0.7491756 , 0.75278236, 0.75645782, 0.75982413,
       0.76322479, 0.76635065, 0.76982001, 0.77356417, 0.77669002,
       0.77967848, 0.78235779, 0.78589585, 0.78905606, 0.79249107,
       0.79513603, 0.79802143, 0.80107859, 0.80358615, 0.80674636,
       0.80935697, 0.81196757, 0.81492168, 0.81784144, 0.8202116 ,
       0.8230283 , 0.82467711, 0.82684117, 0.82903957, 0.83116928,
       0.83340203, 0.83570349, 0.8383828 , 0.84034075, 0.84250

In [28]:
#### Equal Breath Binning
## 25 th percentile 
num=np.array(num)
per_25 = 3
## 50 th percentile
per_50 = num[abs(cdf - 0.50) < 0.003][0]

## 75 th percentile
per_75 = num[abs(cdf - 0.75) < 0.001][0]


In [29]:
temp = data[data["scale"] == "Total"]
temp = temp[temp["level"] == 1]
temp

zip NAICS.id NAICS.display-label RCPSZFE.display-label  ESTAB  \
0        06001    44-45        Retail trade    All establishments     81   
185      06002    44-45        Retail trade    All establishments     63   
342      06006    44-45        Retail trade    All establishments      1   
350      06010    44-45        Retail trade    All establishments    168   
616      06011    44-45        Retail trade    All establishments      2   
631      06013    44-45        Retail trade    All establishments     11   
673      06016    44-45        Retail trade    All establishments      6   
697      06018    44-45        Retail trade    All establishments     17   
768      06019    44-45        Retail trade    All establishments     78   
944      06020    44-45        Retail trade    All establishments      1   
952      06022    44-45        Retail trade    All establishments      4   
983      06023    44-45        Retail trade    All establishments      8   
1028     06024    44-45        Retail trade    All establishments      2   
1043     06026    44-45        Retail trade    All establishments     13   
1097     06027    44-45        Retail trade    All establishments      1   
1101     06029    44-45        Retail trade    All establishments     30   
1198     06030    44-45        Retail trade    All establishments      3   
1217     06031    44-45        Retail trade    All establishments      5   
1244     06032    44-45        Retail trade    All establishments    190   
1491     06033    44-45        Retail trade    All establishments    106   
1704     06035    44-45        Retail trade    All establishments     31   
1816     06037    44-45        Retail trade    All establishments     72   
1997     06039    44-45        Retail trade    All establishments      8   
2035     06040    44-45        Retail trade    All establishments    261   
2343     06042    44-45        Retail trade    All establishments     80   
2520     06043    44-45        Retail trade    All establishments     14   
2588     06045    44-45        Retail trade    All establishments      2   
2600     06050    44-45        Retail trade    All establishments      1   
2608     06051    44-45        Retail trade    All establishments     88   
2776     06052    44-45        Retail trade    All establishments      4   
...        ...      ...                 ...                   ...    ...   
2209954  99302    44-45        Retail trade    All establishments      1   
2209962  99320    44-45        Retail trade    All establishments     10   
2210007  99321    44-45        Retail trade    All establishments      1   
2210010  99322    44-45        Retail trade    All establishments      1   
2210018  99323    44-45        Retail trade    All establishments      5   
2210035  99324    44-45        Retail trade    All establishments     16   
2210112  99326    44-45        Retail trade    All establishments      6   
2210138  99328    44-45        Retail trade    All establishments     18   
2210202  99330    44-45        Retail trade    All establishments      1   
2210206  99336    44-45        Retail trade    All establishments    350   
2210553  99337    44-45        Retail trade    All establishments     28   
2210636  99338    44-45        Retail trade    All establishments      9   
2210673  99341    44-45        Retail trade    All establishments      6   
2210703  99343    44-45        Retail trade    All establishments      5   
2210731  99344    44-45        Retail trade    All establishments     32   
2210824  99347    44-45        Retail trade    All establishments     11   
2210879  99348    44-45        Retail trade    All establishments      1   
2210887  99349    44-45        Retail trade    All establishments     14   
2210946  99350    44-45        Retail trade    All establishments     26   
2211033  99352    44-45        Retail trade    All establishments    114   
2211263  99353    44-45        Retail trade    All establis

In [30]:
scale_labeling = np.vectorize(lambda x : "tiny" if x <= per_25 else ("small" if x <= per_50 else ("medium" if x < per_75 else "large")))
scale = scale_labeling(temp["ESTAB"])
zips = temp["zip"]
zip2scale = {z : s for z, s in zip(zips, scale)}

In [31]:
scale_labeling = np.vectorize(lambda x : zip2scale.get(x, "small"))
data["city scale"] = scale_labeling(data["zip"])

In [32]:
# data["city scale"].value_counts()
temp = data[data["scale"] == "Total"]
temp = temp[temp["level"] == 1]
temp["city scale"].value_counts()

tiny      9822
large     7406
medium    7225
small     4659
Name: city scale, dtype: int64

In [33]:
data["city scale"].value_counts()

large     1455869
medium     527258
small      128738
tiny       100089
Name: city scale, dtype: int64

In [34]:
zip2tw = np.load('zip2tw.npy')
idx2zip = {i : z  for i,z in enumerate(zips)}
zip2idx = {z : i  for i,z in enumerate(zips)}

In [40]:
# # data["total weight"] = 
# give_weight = np.vectorize(lambda x : zip2tw[x])
# data["total weight"] = give_weight(data["zip"])

In [38]:
zip2vec = np.load("zip2vec.npy")

In [60]:
normalized = zip2vec / (np.sum(zip2vec, axis = 1, keepdims = True) + 1e-3)

In [66]:
# plt.pie(normalized)
temp = np.average(normalized, axis = 0) 

In [71]:
temp = temp / temp.sum()

In [80]:
labels = ['Building material and garden equipment and supplies dealers',
 'Clothing and clothing accessories stores',
 'Electronics and appliance stores',
 'Food and beverage stores',
 'Furniture and home furnishings stores',
 'Gasoline stations',
 'General merchandise stores',
 'Health and personal care stores',
 'Miscellaneous store retailers',
 'Motor vehicle and parts dealers',
 'Nonstore retailers',
 'Sporting goods, hobby, musical instrument, and book stores']

## Pie chart for overall US market segmentation

In [105]:
explode = [0,0,0,0,0,0,0,0,0,0,0,0.1]
color = ["red", "salmon", "orange", "yellow", "yellowgreen", "green", "aquamarine", "cyan", "blue", "violet", "magenta", "purple"]
plt.pie(temp, labels=labels,radius=3, explode = explode, autopct='%1.1f%%',
        shadow=True, startangle=60, colors=color, textprops={'fontsize': 16})
# plt.title("US Overall Market Segmentation")
plt.savefig("pie.png")

In [43]:
# ### Takes long to run!!!!!
# zips = data["zip"].unique()
# zip2tw = {z : 0 for z in zips}
# count = 0 
# for z in zip2tw.keys():
#     df = data[data["zip"] == z]
#     df = df[df["level"] == 1]
#     est = np.array(df["ESTAB"])
#     weight = np.array(df["weight"])
#     zip2tw[z] = est.dot(weight)
#     count += 1
#     if count % 100 == 0:
#         print(count)
# zip2tw


In [45]:
# # data[data["zip"] == "06010"]['RCPSZFE.display-label'][351]
# a = data[data["zip"] == "06010"]
# a = a[a['level'] == 1]

In [676]:
cuda = torch.device('cuda')
idx2zip = {i : z  for i,z in enumerate(zips)}
zip2idx = {z : i  for i,z in enumerate(zips)}
total_weights = np.zeros([len(zips), 1])
for k in zip2tw.keys():
    total_weights[zip2idx[k]] = zip2tw[k]

In [16]:
# np.save("zip2tw.npy", total_weights)

In [713]:
zip2vec = np.zeros([len(zips), 12])
count = 0
for z in zip2tw.keys():
    df = data[data["zip"] == z]
    df = df[df["level"] == 2]
    vec = []
    for name in level2_name:
        sub_df = df[df["NAICS.display-label"] == name]
        if len(sub_df) == 0:
            vec.append(0)
            continue
#         print(sub_df)
        est = torch.from_numpy(sub_df["ESTAB"].values).double().to(cuda)
        weight = torch.from_numpy(sub_df["weight"].values).double().to(cuda)
        val = est.dot(weight)
        vec.append(val)
    zip2vec[zip2idx[z]] = vec
    count += 1

    if count % 100 == 0:
        break
#     est = np.array(df["ESTAB"])
#     weight = np.array(df["weight"])
#     zip2tw[z] = est.dot(weight)
    

In [715]:
zip2vec[0]

array([32., 32., 29.,  0., 16., 10., 16., 78., 47., 56., 56., 16.])

In [716]:
zip2vec[1]

array([24., 35., 12., 44., 44., 28., 24., 24.,  2.,  4., 64., 31.])